In [ ]:
#help('modules')

In [ ]:
#help('logging')

## Let's try out the logging module
Maybe it would be nice to be able to store it back to parquet-storage?

In [ ]:
import os
import logging
import pandas as pd
import ast

### Parameters, change these

In [ ]:
### PARAMETERS ###

# Name the text-log
txtlog_name = 'karakterer'
notbook_num = '1'

#### Set up the loggers location

In [ ]:
txtlog_root = 'log'
txtlog_path = f'{txtlog_root}/{txtlog_name}.log'

# Create the log file, if it does not already exist
if not os.path.isdir(txtlog_root):
    os.makedirs(txtlog_root)
if not os.path.isfile(txtlog_path):
    os.mknod(txtlog_path)

# Initialize the logger object
txtlog = logging.getLogger(f'{txtlog_name}.{txtlog_name}{notbook_num}')
fhandler = logging.FileHandler(filename=txtlog_path, mode='a')


#### A custom child class, which handles exception-formatting

In [ ]:
# Extend the logging formatter to do special things with exceptiontext
class OneLineExceptionFormatter(logging.Formatter):
    def formatException(self, exc_info):
        result = super(OneLineExceptionFormatter, self).formatException(exc_info)
        return repr(result) + '' # or format into one line however you want to

    def format(self, record):
        s = super(OneLineExceptionFormatter, self).format(record)
        if record.exc_text:
            s = s.replace('\n', '')
            s = s.replace('"<', "'<")
            s = s.replace('>"', ">'")
        s = s  + '"],\\'
        return s

#### Rest of the loggers configuration

In [ ]:
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = OneLineExceptionFormatter('["%(asctime)s", "%(name)s", "%(levelname)s", "%(message)s', "%Y-%m-%d %H:%M:%S")
fhandler.setFormatter(formatter)
txtlog.addHandler(fhandler)
txtlog.setLevel(logging.DEBUG) # Debug is the lowest level, meaning all logs will be kept.

#### Some examples

In [ ]:
# Write examples of logging-levels to the log
txtlog.debug('Debug is mostly used during development')
txtlog.info('Just to inform user about what is going on')
txtlog.warning('You should take a look at this')
txtlog.error('This should probably not happen')
txtlog.critical('Shit is serious')

# Cause an error on purpose, first_fail() function should not exist
try:
    first_fail()
except Exception as e:
    # Provide exception info in regular logging-levels
    txtlog.critical('Shit is serious, and might be attached to an error: {0}'.format(e), exc_info = True)
    # Exception-info is automatically included at this level, without the kwarg "exec_info" being set to True
    txtlog.exception('We can also catch exceptions, with this conveniance-level. It falls back to the Error-level: {0}'.format(e))
    # Which means that the exception-level is identical to this line:
    txtlog.error('Just to illustrate that this is the same as the exception-conveniance-level: {0}'.format(e), exc_info = True)

#### Demo that we have some control over which levels are being written to the log

In [ ]:
# If code is not in development anymore, maybe you want the debug level to be discarded:
txtlog.setLevel(logging.INFO) 
# The log-messages from lower levels, than the level set, will not be kept.
# So it might be less expensive to never calculate it in the first place
if txtlog.isEnabledFor(logging.DEBUG):
    txtlog.debug('Only printed if debug-level is enabled.')

### Read the list-parsed log file and try writing it into a pandas dataframe

In [ ]:
# We have parsed the log as a list, so lets try importing it back in
contents = ""
with open(txtlog_path) as f:
    contents = contents + f.read()
contents = ast.literal_eval(contents[:-3])

# And inserting the resulting list into a dataframe, which could theoretically be saved back to the parquet-storage.
conts = pd.DataFrame(contents)
#print(conts[3])
pd.set_option('display.max_colwidth', None)
conts.tail(10)